# Hyperparameter Tuning: MinNDAE
In this *Jupyter Notebook* the goal is to find the *optimal hyperparameters* for the `MinNDAE` model using the Kera's `MNIST` dataset as the baseline/standard dataset.

## Setup
Need to get the necessary packages ...

In [ ]:
# check for colab
if "google.colab" in str(get_ipython()):
  # install colab dependencies
  !pip install git+https://github.com/DiogenesAnalytics/autoencoder

## Get MNIST Data
Wille use `keras.datasets` to get the `MNIST` dataset, and then do some *normalizing* and *reshaping* to prepare it for use in training the *autoencoder*.

In [ ]:
# get necessary libs for data/preprocessing
import tensorflow as tf
from keras.datasets import mnist

# load the data
(x_train, _), (x_test, _) = mnist.load_data()

# preprocess the data (normalize)
x_train = x_train.astype("float32") / 255.
x_test = x_test.astype("float32") / 255.

# add grayscale dimension
x_train = tf.expand_dims(x_train, axis=-1)
x_test = tf.expand_dims(x_test, axis=-1)

# convert to tf datasets
train_ds = tf.data.Dataset.from_tensor_slices((x_train, x_train))
test_ds = tf.data.Dataset.from_tensor_slices((x_test, x_test))

# set a few params
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

# update with batch/buffer size
train_ds = train_ds.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_ds = test_ds.batch(BATCH_SIZE)

## Building Hypermodel
Here we need to define the *function* that will be used to build the *hyper model* for the `MinNDAE` class.

In [ ]:
from autoencoder.model.minimal import MinNDParams, MinNDAE

# define the autoencoder model
def build_autoencoder(hp):
    # get encoding dimension
    code_dim = hp.Int("code_dim", min_value=1, max_value=100, step=1)
    
    # get layer configs
    config = MinNDParams(
        l0={"input_shape": (28, 28, 1)},
        l2={"units": code_dim},
        l3={"units": 28 * 28 * 1},
        l4={"target_shape": (28, 28, 1)},
    )

    # create model
    autoencoder = MinNDAE(config)
        
    # select loss function
    autoencoder.compile(optimizer="adam", loss="mean_squared_error")

    # now return keras model
    return autoencoder.model

## Hyperparameter Search
Now we can begin the *hyperparameter search algorithm*.

In [ ]:
# get hyperparam tools
from keras.callbacks import EarlyStopping
from keras_tuner import GridSearch

# setup tuner
tuner = GridSearch(
    build_autoencoder,
    objective="val_loss",
    max_trials=None,
    directory="autoencoder_tuning/minndae",
    project_name=f"code_dim_vs_mse_space/grid_search_1_100",
    seed=42,
)

# create early stop call backs
stop_early = EarlyStopping(monitor="val_loss", patience=2)

# generate random search space for hyperparameters
tuner.search_space_summary()

# run the hyperparameter search
tuner.search(train_ds, epochs=10, validation_data=test_ds, callbacks=[stop_early])

## Optimal Code Dimension
Now we can find the *optimal code dimension* by finding the *x/y pair* that minimizes an *objective function*.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize

# extract score/encode_dims from each trial
code_dims, scores = zip(
    *sorted(
        ((trial.hyperparameters["code_dim"], trial.score) for trial in tuner.oracle.trials.values()),
        key=lambda items: items[0]
    )
)

# convert to numpy arrays
data_x = np.array(code_dims)
data_y = np.array(scores)

# define the objective function
def objective_function(xy):
    x, y = xy
    # use a weighted sum of squared differences as the objective function
    return np.sum((data_x - x)**2 + (data_y - y)**2)

# use averages as initial guess
initial_guess = [np.mean(data_x), np.mean(data_y)]

# set bounds for x and y
bounds = [(min(data_x), max(data_x)), (min(data_y), max(data_y))]

# find the minimum of both x and y
result = minimize(objective_function, initial_guess, bounds=bounds)

# extract the optimal x, y pair
optimal_x, optimal_y = result.x

# plotting the result
plt.scatter(data_x, data_y, label="Model Scores")
plt.title(f"Performance vs Code Dimension:\n{MinNDAE.__name__} / MNIST")
plt.axvline(x=optimal_x, color="r", linestyle="dashed", linewidth=2, label=f"optimal_code_dim: {int(optimal_x)}")
plt.axvline(x=32, color="y", linestyle="dashed", linewidth=2, label="keras_default: 32")
plt.xlabel("Code Dimension")
plt.ylabel("Loss Metric")
plt.legend()
plt.show()